In [1]:
import os
import sys
sys.path.append('../code')

In [2]:
import requests
import time
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import json
from IPython.display import Markdown, display

import altair as alt

load_dotenv()

# Initialize and configure Gemini API
import google.generativeai as genai
from google.generativeai.types import FunctionDeclaration, Tool
import ast

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash")


from llm_connect import get_response
from context import get_all_candidates

# Load in Data

In [6]:
with open("data/mcp_context.json", "r") as f:
    mcp_context = json.load(f)

# Extract candidates section and convert to DataFrame
candidates = list(mcp_context["candidates"].values())
df = pd.DataFrame(candidates)

# Clean and convert fields
df["Years of Experience"] = df["Years of Experience"].astype(int)
df["avg_score"] = df["avg_score"].astype(float)

# Convert Key Skills from stringified list to real list
df["Key Skills"] = df["Key Skills"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Now df is ready for use by your tools
df.head()

,Name,Email,Years of Experience,Key Skills,Llama Score,Candidate ID,Application ID,Resume File,Llama Summary,Gemini Summary,Gemini Score,avg_score,Note,candidate_id,location,education,job_id,application_date,source
0,Katie Mckenzie,katiemckenzie@example.com,9,"[Java, Swift, Kotlin, Python, React Native, AW...",6,b823daf8-4d51-4dd0-adcb-8f990e77dd40,5e801293-f233-4e61-acfc-07edc8525b78,b823daf8-4d51-4dd0-adcb-8f990e77dd40.pdf,Katie Mckenzie is a highly experienced Staff S...,Katie McKenzie presents a mixed profile for th...,6,6.0,Below threshold,b823daf8-4d51-4dd0-adcb-8f990e77dd40,"Chicago, IL",PhD,c143e4c4-0444-45b9-b03f-31c48a310cd0,2025-04-11,Referral
1,Jorge Padilla,jorgepadilla@example.com,3,"[Cloud Infrastructure, Backend Development, No...",6,710b3861-c06c-41bc-8df1-b05524abc855,e528bf36-6672-4a57-8c54-c7984fed1c98,710b3861-c06c-41bc-8df1-b05524abc855.pdf,Jorge Padilla is a mid-level software engineer...,"Jorge Padilla, while holding a PhD in Computer...",4,5.0,Below threshold,710b3861-c06c-41bc-8df1-b05524abc855,"Seattle, WA",PhD,c143e4c4-0444-45b9-b03f-31c48a310cd0,2025-05-05,Company Website
2,Alexander Prince,alexanderprince@example.com,8,"[Cloud Infrastructure, Security Protocols, Pyt...",9,c95788ab-42df-4277-b45c-102d4529df36,37f26b6e-3efd-40a0-870b-66ac104c1c68,c95788ab-42df-4277-b45c-102d4529df36.pdf,Alexander Prince is an exceptionally strong ca...,Alexander Prince presents a strong profile wit...,7,8.0,NaN,c95788ab-42df-4277-b45c-102d4529df36,"New York, NY",Bachelor's Degree,c143e4c4-0444-45b9-b03f-31c48a310cd0,2025-04-01,Company Website
3,Dawn Osborn,dawnosborn@example.com,9,"[Java, Python, JavaScript, C++, Cloud Infrastr...",8,40157b49-1976-4efb-bd27-3e7bf93c94e3,6d4e867b-376d-44aa-a9cc-91d3f92c9bf1,40157b49-1976-4efb-bd27-3e7bf93c94e3.pdf,Dawn Osborn is a highly skilled and experience...,"Dawn Osborn presents a solid profile, exceedin...",7,7.5,NaN,40157b49-1976-4efb-bd27-3e7bf93c94e3,"Chicago, IL",Bachelor's Degree,c143e4c4-0444-45b9-b03f-31c48a310cd0,2025-03-14,Referral
4,Cindy Lambert,cindylambert@example.com,7,"[Java, Python, C++, JavaScript, AWS, Azure, GC...",9,096946f9-8ffa-4238-9297-9c0052599845,1e7f4d54-82e5-4ab0-9b01-92c5b67a516b,096946f9-8ffa-4238-9297-9c0052599845.pdf,Cindy Lambert is an exceptionally qualified ca...,Cindy Lambert presents a compelling profile as...,9,9.0,NaN,096946f9-8ffa-4238-9297-9c0052599845,"San Francisco, CA",PhD,c143e4c4-0444-45b9-b03f-31c48a310cd0,2025-03-08,Indeed


In [13]:
df.dtypes

Name                    object
Email                   object
Years of Experience      int64
Key Skills              object
Llama Score              int64
Candidate ID            object
Application ID          object
Resume File             object
Llama Summary           object
Gemini Summary          object
Gemini Score             int64
avg_score              float64
Note                    object
candidate_id            object
location                object
education               object
job_id                  object
application_date        object
source                  object
dtype: object

# Correlations

In [24]:
# Define a helper function to compute correlation between experience and score
def correlate_experience_with_score() -> dict:
    """Return correlation between experience and score"""
    return df[["Years of Experience", "avg_score"]].corr().to_dict()

# Define tool schema for Gemini to access this function
correlation_params = {
    "type": "object",
    "properties": {},
}
correlation_func = FunctionDeclaration(
    name="correlate_experience_with_score",
    description="Get correlation between experience and candidate score.",
    parameters=correlation_params
)

# Register the tool for Gemini to use
correlation_tool = Tool(function_declarations=[correlation_func])


In [25]:
chat = model.start_chat()

# Ask Gemini to analyze correlation using the defined tool
prompt = (
    "You are a people data analyst assistant. You have access to a tool that can compute the correlation "
    "between candidate experience and their average score. Then explain your findings and opinions in detail."
)

# Send the request and let Gemini call the correlation tool
response = chat.send_message(prompt, tools=[correlation_tool])

# If Gemini called the tool, compute correlation manually and pass it back for interpretation
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    if function_call.name == "correlate_experience_with_score":
        result = correlate_experience_with_score()
        prompt += f"\nTool `correlate_experience_with_score` returned: {json.dumps(result)}"
        response = chat.send_message(prompt)
        display(Markdown(response.text))

Okay, I've analyzed the data regarding candidate experience (measured in years) and their average score based on the tool's output.

**Findings:**

The correlation coefficient between "Years of Experience" and "avg_score" is 0.8653. This is a significant positive correlation.

**Explanation of the Findings:**

*   **Correlation Coefficient:** The correlation coefficient ranges from -1 to +1. A value close to +1 indicates a strong positive correlation, meaning that as one variable increases, the other tends to increase as well. A value close to -1 indicates a strong negative correlation, meaning as one variable increases, the other tends to decrease. A value close to 0 indicates a weak or no linear correlation.
*   **Positive Correlation (0.8653):** In this case, the 0.8653 value suggests a strong positive linear relationship between a candidate's years of experience and their average score. This means that, generally, candidates with more years of experience tend to have higher average scores in the assessment or evaluation process.

**Detailed Opinions and Implications:**

1.  **Experience Matters (But Isn't Everything):** The strong positive correlation indicates that experience is a significant predictor of a candidate's performance (as measured by the "avg\_score"). This is not surprising, as experienced individuals are likely to have developed valuable skills, knowledge, and problem-solving abilities relevant to the role. However, the correlation isn't perfect (it's not 1.0). This means other factors *also* contribute to the average score, such as innate aptitude, learning agility, specific domain knowledge not necessarily tied to overall experience, cultural fit, and even luck during the evaluation process.

2.  **Potential Use in Screening/Selection:** This correlation could be used to inform the initial screening or filtering of candidates.  For instance, if a high average score is crucial for a particular role, prioritizing candidates with a certain minimum level of experience might be a valid strategy. *However, I strongly advise against solely relying on experience*.  It's important to avoid unintentionally discriminating against less experienced but potentially highly capable individuals. Consider the following:

    *   **Define "Experience" Clearly:**  What type of experience is most relevant?  General work experience might not be as valuable as experience specifically in the relevant industry or with a particular technology.
    *   **Look Beyond Years:**  Focus on demonstrable skills and accomplishments. A less experienced candidate who has demonstrably delivered significant results in a related area could be a better fit than a more experienced candidate with a less impressive track record.

3.  **Further Investigation Needed:** While the correlation is strong, I would recommend digging deeper to understand *why* this correlation exists. Are there specific types of experience that are more strongly correlated with success? Are there certain assessment components where experience plays a more significant role? Analyzing these nuances could help refine the hiring process further.

4.  **Consider the "avg_score" Metric:** It's important to understand how the "avg\_score" is calculated.  Is it based on standardized assessments, interviews, portfolio reviews, or a combination of factors?  The nature of the score will influence how much weight should be given to experience. For example, if the average score is heavily weighted towards a technical assessment, experience might be a very reliable predictor. If it's more subjective, the relationship might be less reliable.

5.  **Bias Mitigation:** Be aware of potential biases related to experience. For example, if the company culture historically favors a specific demographic, the "years of experience" metric might inadvertently perpetuate that bias. Ensure that the evaluation process is fair and equitable for all candidates, regardless of their background or years of experience.

**Recommendations:**

*   Use the correlation as a guide, not a rigid rule.
*   Investigate the reasons behind the correlation.
*   Thoroughly define the relevant types of experience.
*   Prioritize skills and demonstrable achievements.
*   Continuously evaluate the fairness and effectiveness of the hiring process.
*   Look at other variables that contribute to higher scores.


## Correlation Analysis: Experience vs. Candidate Scores

This section investigates the relationship between a candidate's years of experience and their evaluated fit for a role, as measured by the average of LLM-generated scores (LLama and Gemini). The goal is to explore whether more experience consistently correlates with higher suitability ratings, or if other factors are more predictive of candidate quality.

### Method

- We compute the **Pearson correlation coefficient** between `Years of Experience` and `Average Score` using Pandas' `.corr()` method.
- This gives a value between -1 and 1, where:
  - `1` indicates a perfect positive correlation
  - `0` means no correlation
  - `-1` indicates a perfect negative correlation

### LLM-Augmented Interpretation

To enhance explainability, the notebook uses **Google's Gemini model** in a tool-calling setup. The LLM:
- Invokes a function to compute the correlation
- Then provides a narrative explanation of the result, offering insights such as:
  - Whether experience is a strong predictor of score
  - Possible reasons for weak or inverse relationships (e.g., skill relevance, recency, resume strength)

This allows for **both statistical precision and narrative interpretation**, enabling a more human-readable understanding of the data.

### Use Case

These findings help inform:
- Whether to weigh experience more heavily during candidate evaluation
- Whether the scoring logic used by the LLMs might be favoring other traits (e.g., skill diversity, project work)

### Relevance

The main point of this section is generate basic insights that recruiters can use based off of the applications they have collected and the insights GenAI has generated so far. At large firms, we can expect these insights can be easily generated amongst people analytics or recruitment analyst teams. Moreover, with the domain expertise each analyst has, it can create statistical insights that match their needs. Nevertheless, in the case that recruitment teams do not have data scientists/analysts that are dedicated for recruitment, they can utilize this AI tool powered by a statistical tool to generate insights. We also allow the LLM to generate explanatory insights for teams to intake. These explanations are tied directly to recruitment use cases, so not only will be generating explanations of the stats, we see appropriate interpretations of the data.

# Chart Generation

In [26]:
# Function to generate various chart types using Altair
def generate_plot(x: str, y: str = None, chart_type: str = "scatter"):
    base = alt.Chart(df).properties(width=600, height=400).interactive()

    if chart_type == "scatter":
        chart = base.mark_point(filled=True, size=80, color="steelblue").encode(
            x=alt.X(x, title=x.title()),
            y=alt.Y(y, title=y.title()),
            tooltip=[x, y]
        ).properties(title=f"Scatter Plot: {y.title()} vs {x.title()}")

    elif chart_type == "bar":
        chart = base.mark_bar(color="seagreen").encode(
            x=alt.X(x, title=x.title()),
            y=alt.Y(y, title=y.title()),
            tooltip=[x, y]
        ).properties(title=f"Bar Chart: {y.title()} vs {x.title()}")

    elif chart_type == "line":
        chart = base.mark_line(strokeWidth=3, color="orangered").encode(
            x=alt.X(x, title=x.title()),
            y=alt.Y(y, title=y.title()),
            tooltip=[x, y]
        ).properties(title=f"Line Chart: {y.title()} vs {x.title()}")

    elif chart_type == "histogram":
        chart = base.mark_bar(color="purple").encode(
            x=alt.X(x, bin=True, title=f"{x.title()} (binned)"),
            y=alt.Y("count()", title="Count"),
            tooltip=[x]
        ).properties(title=f"Histogram: Distribution of {x.title()}")

    else:
        raise ValueError(f"Unsupported chart type: {chart_type}")

    display(chart)


In [27]:
# Define input schema for Gemini to generate charts
plot_params = {
    "type": "object",
    "properties": {
        "x": {"type": "string", "description": "Column for x-axis"},
        "y": {"type": "string", "description": "Column for y-axis"},
        "chart_type": {
            "type": "string",
            "enum": ["scatter", "bar", "line", "histogram"],
            "description": "Type of chart to render"
        }
    },
    "required": ["x"]
}

# Register plot function as a tool for Gemini
plot_func = FunctionDeclaration(
    name="generate_plotly_chart",
    description="Render a chart using Altair based on selected x, y (or only x for histograms), and chart type.",
    parameters=plot_params
)

plot_tool = Tool(function_declarations=[plot_func])


In [28]:
# Function to interpret Gemini's function call and generate the chart
def try_generate_plot_and_assess(response, label="Plot", ask_for_assessment=False, chat=None):
    try:
        if not response.candidates:
            print(f"{label}: No candidates returned by Gemini.")
            return

        parts = response.candidates[0].content.parts
        if not parts:
            print(f"{label}: No content parts returned by Gemini.")
            return

        plot_rendered = False
        for part in parts:
            if hasattr(part, "function_call") and part.function_call is not None:
                fn = part.function_call
                args = dict(fn.args) if fn.args else {}
                x = args.get("x")
                y = args.get("y", None)
                chart_type = args.get("chart_type", "scatter")

                # Validate column names
                if x not in df.columns:
                    print(f"{label}: Invalid x column: {x}")
                    continue
                if chart_type != "histogram" and (y is None or y not in df.columns):
                    print(f"{label}: Invalid or missing y column: {y}")
                    continue

                # Call charting function
                print(f"{label}: Generating {chart_type} chart of {y} vs {x}")
                generate_plot(x=x, y=y, chart_type=chart_type)
                plot_rendered = True

        # Ask for LLM-generated assessment of the chart if requested
        if ask_for_assessment and chat is not None:
            columns = [x] + ([y] if y else [])
            summary = df[columns].describe().to_string()
            sample = df[columns].sample(10).to_string(index=False)

            followup = chat.send_message(
                "Here is the summary and sample data from the chart you just created \n\n"
                f"Summary: {summary}\n\n"
                f"Sample Rows: \n{sample}\n\n"
                "Now give a recruiter-friendly assessment of the chart you just created."
                "Explain what insights it reveals, trends, outliers, and how it might inform hiring decisions."
                "Be detailed and be clear of why the chart shapes up the way it did."
            )

            display(Markdown(followup.text.strip()))

        if not plot_rendered:
            print(f"{label}: No chart generated.")

    except Exception as e:
        print(f"{label}: Failed with error: {e}")

# Create textual summary and sample rows to give LLM context
sample_rows = df.head(5).to_string(index=False)
summary = df.describe(include='all').to_string()


chat = model.start_chat()

# Send structured prompt and allow Gemini to generate a chart and analyze it
prompt = (
    "Here is some information about the data you are going to use:\n"
    f"Summary: {summary}\n\n"
    f"Sample Rows: {sample_rows}\n\n"
    f"Data Types: {df.dtypes}\n\n"
    "Use the `plot_tool` tool to create a chart that helps recruiters understand candidate data."
    "Do NOT describe the chart. Use the tool directly. Then give an assessment in a follow-up."
    f" Available columns: {list(df.columns)}."
)

response1 = chat.send_message(prompt, tools=[plot_tool])
try_generate_plot_and_assess(response1, label="Plot + Assessment 1", ask_for_assessment=True, chat=chat)

prompt2 = 'Make me a second unique chart, \n' + prompt
response2 = chat.send_message(prompt2, tools=[plot_tool])
try_generate_plot_and_assess(response2, label='Plot + Assessment 2', ask_for_assessment=True, chat=chat)

prompt3 = 'Make me a third unique chart, \n' + prompt
response3 = chat.send_message(prompt3, tools=[plot_tool])
try_generate_plot_and_assess(response3, label='Plot + Assessment 2', ask_for_assessment=True, chat=chat)


Plot + Assessment 1: Generating scatter chart of avg_score vs Years of Experience


alt.Chart(...)

The scatter plot of "Years of Experience" vs. "avg_score" visualizes the relationship between a candidate's experience and their average score, offering insights into how experience correlates with overall candidate assessment.

**Insights and Trends:**

*   **Positive Correlation:** There appears to be a generally positive, albeit not perfectly linear, correlation between years of experience and the average score. This suggests that, on average, candidates with more experience tend to receive higher scores.
*   **Score Variation at Lower Experience:** The data points at the lower end of the "Years of Experience" spectrum (e.g., 0-3 years) show a wider range of average scores. This could mean that factors other than just experience play a significant role in evaluating less experienced candidates, such as their skills, education, or specific projects.
*   **Experience Plateau:** As years of experience increase, the rise in average score might plateau. This could indicate that after a certain point, additional years of experience don't necessarily translate into a significantly higher assessment, or that other factors become more important at senior levels.

**Outliers:**

*   **High Score, Low Experience:** There are a few data points where candidates with relatively low years of experience have achieved high average scores. These could be exceptional junior candidates who possess outstanding skills or demonstrate high potential.
*   **Low Score, High Experience:** Conversely, there might be a few candidates with a high number of years of experience but relatively low average scores. These could be candidates whose skills don't align well with the job requirements, or who haven't kept their skills up to date.

**Informing Hiring Decisions:**

*   **Experience as a Guide:** Recruiters can use this chart to understand the general trend and set expectations. It can act as a starting point for filtering candidates based on experience.
*   **Holistic Assessment:** The scatter plot highlights that experience is not the only factor. Recruiters should always consider other factors, such as key skills, education, and the summaries from the LLM's.
*   **Identifying Hidden Gems:** Outliers (high score, low experience) are candidates that deserve a closer look as these could be exceptionally talented individuals.
*   **Addressing Potential Concerns:** Outliers (low score, high experience) may require deeper investigation to understand the discrepancy.

**Why the Chart Shapes Up This Way:**

*   The positive correlation reflects the common expectation that more experienced professionals usually have a more developed skill set and a proven track record.
*   The variation at the lower experience levels may stem from the fact that junior roles vary significantly in complexity, and candidates might come from diverse backgrounds.
*   The plateau effect could be because, beyond a certain experience threshold, factors like leadership, specialization, or adaptability become more critical than sheer years of service.

Overall, this chart is a visual aid in understanding the relationship between experience and candidate quality, enabling recruiters to make more informed decisions.

Plot + Assessment 2: Generating bar chart of avg_score vs education


alt.Chart(...)

The bar chart visualizes the relationship between a candidate's education level and their average score. It allows recruiters to quickly compare the average scores of candidates with different education backgrounds.

**Insights and Trends:**

*   **Average Score by Education Level:** The chart displays the average score for each education category (Bachelor's Degree, Master's Degree, and PhD). This allows for a direct comparison of how candidates with different levels of education are generally assessed.
*   **Potential Score Distribution:** While the chart shows the average, it's important to remember that there will be a distribution of scores within each education category. Some candidates with a Bachelor's degree might score higher than some with a Master's or PhD, and vice versa.

**Interpreting the Chart's Shape and Outliers:**

*   If one education level has a significantly higher average score than others, it *could* suggest that candidates with that level of education are, on average, a stronger fit for the role. However, it is vital to consider that this might be correlated with the job description.
*   Outliers in this context aren't directly visible in the bar chart, but they would be candidates whose individual scores deviate significantly from the average for their education level.
*   A bachelor's degree can be very focused. A master's degree may show specialized and advanced knowledge in a certain field, which may align well with the job. A Ph.D. may show research capabilities, problem solving skills, and subject matter expertise. Depending on the role, any could be a great fit.

**Informing Hiring Decisions:**

*   **Education as a Factor:** The chart provides a quick overview of how education level relates to average candidate assessment, but it shouldn't be the sole determining factor.
*   **Consider the Role Requirements:** The relevance of education level depends heavily on the specific requirements of the role. Some roles may require advanced degrees, while others may prioritize experience and skills over formal education.
*   **Further Investigation:** The chart prompts further investigation. If one education level consistently has lower scores, it might indicate a misalignment between the skills acquired through that education and the requirements of the job. However, this should be validated by looking at individual candidate profiles.
*   **Holistic Review:** Use this as a single factor, combined with skillsets, experience, and the AI summaries to make an informed decision.

The bar chart helps recruiters quickly assess the general trend of candidate scores across different education levels, enabling them to factor education into their overall evaluation process while still maintaining a holistic approach.

Plot + Assessment 2: Generating histogram chart of None vs location


alt.Chart(...)

The histogram visualizes the distribution of candidates across different locations. This is a useful tool for understanding where the applicant pool is concentrated.

**Insights and Trends:**

*   **Candidate Distribution:** The chart clearly shows the number of candidates from each listed location. This allows the recruiter to see at a glance which locations are producing the most applicants.
*   **Concentration of Applicants:** Some locations may have significantly higher numbers of applicants than others. This can indicate where the company's recruitment efforts are most effective, or where there's a higher concentration of talent relevant to the job.

**Interpreting the Chart's Shape:**

*   The height of each bar represents the frequency of candidates from a specific location. A taller bar indicates more candidates from that location.
*   The relative heights of the bars give a quick visual comparison of the candidate pool across different locations.

**Informing Hiring Decisions:**

*   **Targeted Recruitment:** If the company is looking to increase diversity of location within its workforce, this chart can highlight areas where recruitment efforts may need to be strengthened. It can inform decisions to target recruitment marketing in specific geographic areas.
*   **Relocation Considerations:** The recruiter can consider if the role has any relocation opportunities and how that fits in with where the applicant is coming from.
*   **Remote Work Potential:** In a remote-friendly environment, the location of candidates might be less critical. However, understanding where candidates are based can still be relevant for time zone alignment, team collaboration, and potential future in-person meetings.
*   **Cost of Living:** Location can inform salary expectations as costs of living vary significantly across locations.

**Why the Chart Shapes Up This Way:**

*   **Company Reputation:** If the company has a strong reputation in a particular city, it's likely to attract more candidates from that location.
*   **Local Talent Pool:** Some locations have larger pools of talent for specific industries or job functions.
*   **Recruitment Efforts:** The company's recruitment efforts (e.g., job postings, career fairs) in different locations will directly impact the number of applicants from those areas.

Overall, this chart provides a valuable snapshot of the geographic distribution of candidates, enabling recruiters to make informed decisions about targeted recruitment strategies, remote work considerations, and diversity initiatives.

## Visualization and Chart Generation

This section provides visual insight into candidate evaluation data by generating charts that explore relationships between key variables such as years of experience, average score, education level, and application source.

### Objectives

- Summarize the candidate level data through visualizations
- Highlight patterns that can be uncovered by data visualizations
- Allow charts to be generated automatically by GenAI with tool calling

### Implementation

Using `altair`, AI can generate charts such as:

- **Histogram of Average Scores**: Visualizes how candidates are distributed based on their combined LLama and Gemini evaluation.
- **Scatter Plot: Experience vs. Score**: Helps identify whether there’s a visible linear or nonlinear relationship between experience and perceived fit.
- **Bar Charts by Education Level or Source**: Shows how different candidate segments perform on average or how they're represented in the applicant pool.

These visualizations serve as a bridge between raw data and human intuition, allowing hiring managers or analysts to grasp broader trends at a glance. The choice to use altair is to utilize the `JavaScript` integrations to allow charts to be interactable via tooltips and adjustable charts.

To ensure the charts being generated are "appropriate" we expose the LLM to the data that has been generated before. So by passing the data types, summaries, and few rows of the data, Gemini will use its own judgment to choose which variables and which chart type it will create. It is still susceptible to poor variable choices that lead to bad charts. But to prevent that, it is necessary to have many features it can choose from.

But to create actual insights, we need to expose Gemini to as much data as possible. In an API integration of Gemini, it does not have access to computer vision. Hence, it must be exposed to charts directly from the data itself through string formatting or through HTML version of the chart. But as the complexity of the chart can scale with more features or data, it is best to expose it to summary statistics via the `describe()` function. Moreover, we provide it with a sample of the data to provide the LLM with a Stochastic view of the data. This will be the best when there is a large dataset; it assumes that a small sample will be representative of the general population, so even a small subset will be enough to generate insights upon. We also do prompt the LLM to be very detailed in the insights it creates. Allowing it to explore different perspectives will create for richer information amongst recruitment teams, hopefully leading to more informed decisions.

### Use Case

These charts support:
- Stakeholder reporting
- Bias detection (e.g., whether certain education levels are consistently scored lower)
- Informing adjustments to evaluation criteria or LLM prompting strategies

### Relevance

The importance of this section is to again implement tool calling by LLMs to create data insights for recruitment. As stated before, the point of these tools is to generate data science insights for recruitment teams that do not have time and resources to implement data science strategies for recruitment. Generally, speaking we expect data scientists to gain an understanding of the data before creating insights. This may require a couple of hours to couple of days (depending on the size of the data). However, by allowing LLMs to understand the data by passing it through its chat interface, it should have a good understanding of the data at a faster rate than one data analyst that has been newly introduced to a dataset. This not only will save time, it allows smaller firms to implement recruitment data analyst strategies that managers can use to make smarter hiring decisions. Moreover, similar to the statistical analysis that was implemented to before, we will also allow the LLM to generate insights of the chart it created. Although it is not necessarily hard to interpret 2D charts, we can allow LLMs to catch nuances and unique insights that a novice or tunnel visioned data analyst may have.

The key difference between the first and second implementation of data analytics tools is the amount of freedom Gemini is allowed to have when allowing it to choose the chart type and variables it uses. However, issue is that if the LLM choose "bad" variables or chart types, it is hard to control without explicit prompting. For example, the LLM may choose to plot a histogram of "Skills" that were listed on a resume. What happens to be generated is a very messy graph that does not produce any insights. Although it is possible to prevent these types of mistakes, it is very much dependent on the prompting that Gemini receives. But in order to prevent a narrow pipeline of chart generation, we keep the prompt very loose. But by providing it as much context as possible, we can hope that the LLM will be smart in its choices.